In [1]:
import pandas as pd
import numpy as np

In [2]:
df_train=pd.read_parquet('train_data.parquet')
df_test=pd.read_parquet('test_data.parquet')

In [3]:
df_offer=pd.read_parquet('offer_metadata.parquet')
df_transaction=pd.read_parquet('add_trans.parquet')
df_event=pd.read_parquet('add_event.parquet')

In [4]:
df_transaction.columns

Index(['id2', 'f367', 'f368', 'f369', 'f370', 'f371', 'f372', 'id8', 'f374'], dtype='object')

In [5]:
df_offer.head()

,id3,id9,f375,f376,f377,id10,id11,f378,f374,id8,id12,id13
0,70687,FO5O,2,5.0,None,1,None,N,None,None,2018-01-01 00:00:00,2099-12-31 23:59:59
1,900002526,UGE,2,100.0,None,1,None,N,None,None,2014-10-20 00:00:00,2099-12-31 23:59:59
2,900002864,UTP,1,100.0,None,1,None,N,None,None,2016-07-19 00:00:00,2099-12-31 23:59:59
3,19508,o,2,NaN,None,1,None,N,None,None,2019-06-02 17:00:00,2028-12-31 16:59:59
4,35903,o,2,NaN,None,1,None,N,None,None,2019-06-02 17:00:00,2028-12-31 16:59:59


In [6]:
df_offer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4164 entries, 0 to 4163
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id3     4164 non-null   int32  
 1   id9     4164 non-null   object 
 2   f375    4164 non-null   int32  
 3   f376    978 non-null    float64
 4   f377    0 non-null      object 
 5   id10    4164 non-null   object 
 6   id11    0 non-null      object 
 7   f378    4164 non-null   object 
 8   f374    3887 non-null   object 
 9   id8     3887 non-null   object 
 10  id12    4164 non-null   object 
 11  id13    4164 non-null   object 
dtypes: float64(1), int32(2), object(9)
memory usage: 358.0+ KB


In [7]:
def summarize_dataframe(df, name="DataFrame"):
    print(f"\n===== Summary of {name} =====")
    summary = []
    for col in df.columns:
        dtype = df[col].dtype
        unique_vals = df[col].nunique(dropna=False)
        sample_vals = df[col].dropna().unique()[:5]  # show first 5 unique (non-null) values
        summary.append((col, dtype, unique_vals, sample_vals))
    
    summary_df = pd.DataFrame(summary, columns=["Column", "Data Type", "Unique Values", "Sample Unique Values"])
    return summary_df

# Example usage:



In [8]:

summarize_dataframe(df_offer, "Offer Metadata")


===== Summary of Offer Metadata =====


,Column,Data Type,Unique Values,Sample Unique Values
0,id3,int32,4164,"[70687, 900002526, 900002864, 19508, 35903]"
1,id9,object,1614,"[FO5O, UGE, UTP, o, FS5O]"
2,f375,int32,2,"[2, 1]"
3,f376,float64,22,"[5.0, 100.0, 1.0, 7.1, 40.0]"
4,f377,object,1,[]
5,id10,object,2,"[1, 2]"
6,id11,object,1,[]
7,f378,object,1252,"[N, gao$scbuyeectsamo$ioomporrarcwypfysalnypf1..."
8,f374,object,134,"[Grocery Stores, Hotels & Motels, Eating Place..."
9,id8,object,301,"[54110000, 70110000, 58120000, 54110100, 43110..."


In [9]:
df_offer=df_offer.drop(columns=['f377','id11','f378'])

In [10]:
summarize_dataframe(df_offer, "Offer Metadata")


===== Summary of Offer Metadata =====


,Column,Data Type,Unique Values,Sample Unique Values
0,id3,int32,4164,"[70687, 900002526, 900002864, 19508, 35903]"
1,id9,object,1614,"[FO5O, UGE, UTP, o, FS5O]"
2,f375,int32,2,"[2, 1]"
3,f376,float64,22,"[5.0, 100.0, 1.0, 7.1, 40.0]"
4,id10,object,2,"[1, 2]"
5,f374,object,134,"[Grocery Stores, Hotels & Motels, Eating Place..."
6,id8,object,301,"[54110000, 70110000, 58120000, 54110100, 43110..."
7,id12,object,669,"[2018-01-01 00:00:00, 2014-10-20 00:00:00, 201..."
8,id13,object,672,"[2099-12-31 23:59:59, 2028-12-31 16:59:59, 209..."


In [11]:
df_offer['id12'] = pd.to_datetime(df_offer['id12'], errors='coerce')
df_offer['id13'] = pd.to_datetime(df_offer['id13'], errors='coerce')


In [12]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# 1. Duration of the offer in days
df_offer['offer_duration_days'] = (df_offer['id13'] - df_offer['id12']).dt.days

# 2. Is offer active now?
now = pd.Timestamp.now()
df_offer['is_active_now'] = ((now >= df_offer['id12']) & (now <= df_offer['id13'])).astype(int)

# 3. Month in which offer started
df_offer['start_month'] = df_offer['id12'].dt.month

# # 4. Label Encode: Redemption Frequency (f375)
# le_freq = LabelEncoder()
# df_offer['redemption_freq_enc'] = le_freq.fit_transform(df_offer['f375'].astype(str))

# # 5. Label Encode: Offerings Body (id10)
# le_body = LabelEncoder()
# df_offer['offer_body_enc'] = le_body.fit_transform(df_offer['id10'].astype(str))

# # 6. Label Encode Industry Name (f374)
# le_industry = LabelEncoder()
# df_offer['industry_name_enc'] = le_industry.fit_transform(df_offer['f374'].astype(str))

# 7. Select only engineered and relevant features
offer_features = df_offer[[
    'id3',                     # Offer ID for merging
    'offer_duration_days',
    'is_active_now',
    'start_month',
    'f376',                   # Discount rate (numeric)
    'f374'
]]


In [13]:
summarize_dataframe(offer_features)


===== Summary of DataFrame =====


,Column,Data Type,Unique Values,Sample Unique Values
0,id3,int32,4164,"[70687, 900002526, 900002864, 19508, 35903]"
1,offer_duration_days,int64,284,"[29949, 31118, 30480, 3499, 29316]"
2,is_active_now,int64,2,"[1, 0]"
3,start_month,int32,12,"[1, 10, 7, 6, 9]"
4,f376,float64,22,"[5.0, 100.0, 1.0, 7.1, 40.0]"
5,f374,object,134,"[Grocery Stores, Hotels & Motels, Eating Place..."


In [14]:
import pandas as pd

# Assumes 'id12' and 'id13' are already datetime
now = pd.Timestamp.now()

# 1. is_high_discount (compared to median across all offers)
df_offer['is_high_discount'] = (df_offer['f376'] > df_offer['f376'].median()).astype(int)

# 2. is_limited_duration (e.g., offers < 7 days long)
df_offer['offer_duration_days'] = (df_offer['id13'] - df_offer['id12']).dt.days
df_offer['is_limited_duration'] = (df_offer['offer_duration_days'] < 7).astype(int)

# 3. Seasonality flags (e.g., offers starting in festive or seasonal months)
# You can modify months based on country/region's holidays
festive_months = [10, 11, 12]  # e.g., Diwali, Christmas, New Year
df_offer['start_month'] = df_offer['id12'].dt.month
df_offer['is_festive_offer'] = df_offer['start_month'].isin(festive_months).astype(int)

# 4. Offer competitiveness (discount vs. average discount of same industry)
# # Use f374 as the industry name column
# industry_avg_discount = df_offer.groupby('f374')['f376'].transform('mean')
# df_offer['offer_competitiveness'] = df_offer['f376'] - industry_avg_discount


In [15]:
summarize_dataframe(offer_features)


===== Summary of DataFrame =====


,Column,Data Type,Unique Values,Sample Unique Values
0,id3,int32,4164,"[70687, 900002526, 900002864, 19508, 35903]"
1,offer_duration_days,int64,284,"[29949, 31118, 30480, 3499, 29316]"
2,is_active_now,int64,2,"[1, 0]"
3,start_month,int32,12,"[1, 10, 7, 6, 9]"
4,f376,float64,22,"[5.0, 100.0, 1.0, 7.1, 40.0]"
5,f374,object,134,"[Grocery Stores, Hotels & Motels, Eating Place..."


In [16]:
# List of newly created columns to add
new_cols = [
    'is_high_discount',
    'is_limited_duration',
    'is_festive_offer',
]

# Make sure id3 is included if it's not already in offer_features
if 'id3' not in offer_features.columns:
    offer_features = df_offer[['id3'] + new_cols]
else:
    # Merge new features into existing offer_features by id3
    offer_features = offer_features.merge(df_offer[['id3'] + new_cols], on='id3', how='left')


In [17]:
summarize_dataframe(offer_features)


===== Summary of DataFrame =====


,Column,Data Type,Unique Values,Sample Unique Values
0,id3,int32,4164,"[70687, 900002526, 900002864, 19508, 35903]"
1,offer_duration_days,int64,284,"[29949, 31118, 30480, 3499, 29316]"
2,is_active_now,int64,2,"[1, 0]"
3,start_month,int32,12,"[1, 10, 7, 6, 9]"
4,f376,float64,22,"[5.0, 100.0, 1.0, 7.1, 40.0]"
5,f374,object,134,"[Grocery Stores, Hotels & Motels, Eating Place..."
6,is_high_discount,int64,2,"[0, 1]"
7,is_limited_duration,int64,2,"[0, 1]"
8,is_festive_offer,int64,2,"[0, 1]"


In [18]:
# Step 1: One-hot encode f374
f374_dummies = pd.get_dummies(df_offer['f374'], prefix='industry', drop_first=False)

# Step 2: Concatenate with id3 for merging
f374_encoded = pd.concat([df_offer['id3'], f374_dummies], axis=1)

# Step 3: Merge encoded features into offer_features
offer_features = offer_features.merge(f374_encoded, on='id3', how='left')


In [19]:
def bool_con(df):
    bool_cols = df.select_dtypes(include='bool').columns
    df[bool_cols] = df[bool_cols].astype(int)
    return df

In [20]:
offer_features=bool_con(offer_features)

In [21]:
# Identify all industry dummy columns
industry_cols = [col for col in offer_features.columns if col.startswith('industry_')]

# Drop one (e.g., the first one)
offer_features.drop(columns=[industry_cols[0]], inplace=True)


In [22]:
offer_features=offer_features.drop(columns=['f374'])

In [23]:
offer_features.head()

,id3,offer_duration_days,is_active_now,start_month,f376,is_high_discount,is_limited_duration,is_festive_offer,"industry_Accounting, Auditing, And Bookkeeping Services",industry_Advertising Agencies,...,industry_United States Postal Service,industry_Used Merchandise Stores,industry_Utility Trailer And Recreational Vehicle Rental,industry_Variety Stores,industry_Veterinary Services For Animal Specialties,industry_Video Tape Rental,"industry_Water Transportation Services, Nec",industry_Women's & Children's Clothing,industry_Women's Accessory And Specialty Stores,industry_Women's Clothing Stores
0,70687,29949,1,1,5.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,900002526,31118,1,10,100.0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,900002864,30480,1,7,100.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,19508,3499,1,6,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,35903,3499,1,6,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
# Convert f377 (discount rate) to float
offer_features['f376'] = pd.to_numeric(offer_features['f376'], errors='coerce')


In [25]:
# Identify industry columns
industry_cols = [col for col in offer_features.columns if col.startswith("industry_")]

# Force all industry columns to numeric (in case any are still strings)
offer_features[industry_cols] = offer_features[industry_cols].apply(pd.to_numeric, errors='coerce')


In [26]:
# Step 1: Identify one-hot encoded industry columns
industry_cols = [col for col in offer_features.columns if col.startswith("industry_")]

# Step 2: Force all to numeric (in-place)
for col in industry_cols:
    offer_features[col] = pd.to_numeric(offer_features[col], errors='coerce')

# Step 3: Decode one-hot back to category name
offer_features['industry_decoded'] = (
    offer_features[industry_cols]
    .astype(float)  # ensure all numeric for argmax
    .idxmax(axis=1)
    .str.replace('industry_', '', regex=False)
)

# Step 4: Ensure f376 is also numeric
offer_features['f376'] = pd.to_numeric(offer_features['f376'], errors='coerce')

# Step 5: Compute offer competitiveness
industry_avg_discount = offer_features.groupby('industry_decoded')['f376'].transform('mean')
offer_features['offer_competitiveness'] = offer_features['f376'] - industry_avg_discount

# Optional: Drop the temporary decoded label
offer_features.drop(columns=['industry_decoded'], inplace=True)


C:\Users\91637\AppData\Local\Temp\ipykernel_33900\524678378.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  offer_features['industry_decoded'] = (
C:\Users\91637\AppData\Local\Temp\ipykernel_33900\524678378.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  offer_features['offer_competitiveness'] = offer_features['f376'] - industry_avg_discount


In [27]:
summarize_dataframe(offer_features)


===== Summary of DataFrame =====


,Column,Data Type,Unique Values,Sample Unique Values
0,id3,int32,4164,"[70687, 900002526, 900002864, 19508, 35903]"
1,offer_duration_days,int64,284,"[29949, 31118, 30480, 3499, 29316]"
2,is_active_now,int64,2,"[1, 0]"
3,start_month,int32,12,"[1, 10, 7, 6, 9]"
4,f376,float64,22,"[5.0, 100.0, 1.0, 7.1, 40.0]"
...,...,...,...,...
136,"industry_Water Transportation Services, Nec",int64,2,"[0, 1]"
137,industry_Women's & Children's Clothing,int64,2,"[0, 1]"
138,industry_Women's Accessory And Specialty Stores,int64,2,"[0, 1]"
139,industry_Women's Clothing Stores,int64,2,"[0, 1]"


In [28]:
offer_features['is_uncompetitive'] = (offer_features['offer_competitiveness'] < 0).astype(int)


C:\Users\91637\AppData\Local\Temp\ipykernel_33900\1180865403.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  offer_features['is_uncompetitive'] = (offer_features['offer_competitiveness'] < 0).astype(int)


In [29]:
offer_features['discount_per_day'] = offer_features['f376'] / (offer_features['offer_duration_days'] + 1)


C:\Users\91637\AppData\Local\Temp\ipykernel_33900\956656515.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  offer_features['discount_per_day'] = offer_features['f376'] / (offer_features['offer_duration_days'] + 1)


In [30]:
# How many days ago the offer started
offer_features['days_since_start'] = (pd.Timestamp.now() - df_offer['id12']).dt.days
# How many days left
offer_features['days_until_expiry'] = (df_offer['id13'] - pd.Timestamp.now()).dt.days


C:\Users\91637\AppData\Local\Temp\ipykernel_33900\4203570839.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  offer_features['days_since_start'] = (pd.Timestamp.now() - df_offer['id12']).dt.days
C:\Users\91637\AppData\Local\Temp\ipykernel_33900\4203570839.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  offer_features['days_until_expiry'] = (df_offer['id13'] - pd.Timestamp.now()).dt.days


In [31]:
offer_features['start_month_sin'] = np.sin(2 * np.pi * offer_features['start_month'] / 12)
offer_features['start_month_cos'] = np.cos(2 * np.pi * offer_features['start_month'] / 12)


C:\Users\91637\AppData\Local\Temp\ipykernel_33900\2004245737.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  offer_features['start_month_sin'] = np.sin(2 * np.pi * offer_features['start_month'] / 12)
C:\Users\91637\AppData\Local\Temp\ipykernel_33900\2004245737.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  offer_features['start_month_cos'] = np.cos(2 * np.pi * offer_features['start_month'] / 12)


In [32]:
summarize_dataframe(offer_features)


===== Summary of DataFrame =====


,Column,Data Type,Unique Values,Sample Unique Values
0,id3,int32,4164,"[70687, 900002526, 900002864, 19508, 35903]"
1,offer_duration_days,int64,284,"[29949, 31118, 30480, 3499, 29316]"
2,is_active_now,int64,2,"[1, 0]"
3,start_month,int32,12,"[1, 10, 7, 6, 9]"
4,f376,float64,22,"[5.0, 100.0, 1.0, 7.1, 40.0]"
...,...,...,...,...
142,discount_per_day,float64,302,"[0.0001669449081803005, 0.0032134708698865646,..."
143,days_since_start,int64,300,"[2757, 3926, 3288, 2240, 2124]"
144,days_until_expiry,int64,293,"[27192, 1259, 27167, 163, -661]"
145,start_month_sin,float64,11,"[0.49999999999999994, -0.8660254037844386, -0...."


In [33]:
# High discount * festive
offer_features['high_discount_festive'] = offer_features['is_high_discount'] * offer_features['is_festive_offer']

# Discount competitiveness per day
offer_features['competitiveness_per_day'] = offer_features['offer_competitiveness'] / (offer_features['offer_duration_days'] + 1)


C:\Users\91637\AppData\Local\Temp\ipykernel_33900\2968513222.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  offer_features['high_discount_festive'] = offer_features['is_high_discount'] * offer_features['is_festive_offer']
C:\Users\91637\AppData\Local\Temp\ipykernel_33900\2968513222.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  offer_features['competitiveness_per_day'] = offer_features['offer_competitiveness'] / (offer_features['offer_duration_days'] + 1)


In [34]:
offer_features.shape

(4164, 149)

In [35]:
df_offer['id3'] = df_offer['id3'].astype(str)
offer_features['id3'] = offer_features['id3'].astype(str)


In [36]:
offer_features = pd.merge(
    offer_features,
    df_offer[['id3', 'id8']],  # Only take 'id3' and 'id8' from df_offer
    on='id3',
    how='left'  # Keep all rows in offer_features
)


In [40]:
df_offer.head()

,id3,id9,f375,f376,id10,f374,id8,id12,id13,offer_duration_days,is_active_now,start_month,is_high_discount,is_limited_duration,is_festive_offer
0,70687,FO5O,2,5.0,1,None,None,2018-01-01 00:00:00,2099-12-31 23:59:59,29949,1,1,0,0,0
1,900002526,UGE,2,100.0,1,None,None,2014-10-20 00:00:00,2099-12-31 23:59:59,31118,1,10,1,0,1
2,900002864,UTP,1,100.0,1,None,None,2016-07-19 00:00:00,2099-12-31 23:59:59,30480,1,7,1,0,0
3,19508,o,2,NaN,1,None,None,2019-06-02 17:00:00,2028-12-31 16:59:59,3499,1,6,0,0,0
4,35903,o,2,NaN,1,None,None,2019-06-02 17:00:00,2028-12-31 16:59:59,3499,1,6,0,0,0


In [41]:
offer_features['id8'].unique()



array([None, '54110000', '70110000', '58120000', '54110100', '43110000',
       '58120600', '59990000', '54990000', '59440000', '58130100',
       '55410000', '57310000', '57190203', '56110000', '59610000',
       '59630000', '86990000', '56210000', '44810000', '59639991',
       '59430000', '72310000', '59449901', '58121701', '48131100',
       '79990000', '57190000', '73890700', '56510000', '56219901',
       '56510500', '59129900', '75149902', '59410000', '57190104',
       '53119900', '59129901', '78419902', '47249900', '59991300',
       '47240000', '47850100', '58120310', '59210000', '59619923',
       '57350000', '58120500', '79229909', '50450000', '73890000',
       '59991100', '54219902', '56320204', '59470000', '56990300',
       '56990000', '50650000', '41210000', '58120100', '51220208',
       '54219910', '63210000', '59410305', '78320000', '45120100',
       '53110000', '59639998', '60810000', '59210104', '57340000',
       '50740000', '59320000', '59639997', '81110000', '

In [35]:
# summarize_dataframe(df_event)

In [42]:
event_impressions = df_event.groupby('id3')['id4'].count().reset_index(name='num_impressions')


In [43]:
df_event['clicked'] = df_event['id7'].notna().astype(int)
event_clicks = df_event.groupby('id3')['clicked'].sum().reset_index(name='num_clicks')


In [44]:
event_ctr = pd.merge(event_impressions, event_clicks, on='id3')
event_ctr['ctr'] = event_ctr['num_clicks'] / (event_ctr['num_impressions'] + 1)


In [45]:
df_event['impression_time'] = pd.to_datetime(df_event['id4'], errors='coerce')
df_event['click_time'] = pd.to_datetime(df_event['id7'], errors='coerce')
df_event['click_delay_seconds'] = (df_event['click_time'] - df_event['impression_time']).dt.total_seconds()

event_click_delay = df_event[df_event['click_delay_seconds'].notna()] \
    .groupby('id3')['click_delay_seconds'] \
    .mean().reset_index(name='avg_click_delay_secs')


In [46]:
event_ctr.head()

,id3,num_impressions,num_clicks,ctr
0,10139,336,47,0.139466
1,101460,3277,202,0.061623
2,10242,47995,313,0.006521
3,104246,14157,594,0.041955
4,106207,968,14,0.014448


In [47]:
event_click_delay.head()

,id3,avg_click_delay_secs
0,10139,4.692439
1,101460,11.481268
2,10242,31.791639
3,104246,46.669261
4,106207,36.804567


In [48]:
placement_dummies = pd.get_dummies(df_event['id6'], prefix='placement')
placement_counts = pd.concat([df_event[['id3']], placement_dummies], axis=1)
placement_stats = placement_counts.groupby('id3').sum().reset_index()

In [49]:
event_features = event_ctr.merge(event_click_delay, on='id3', how='left')
event_features = event_features.merge(placement_stats, on='id3', how='left')

In [50]:
event_features['avg_click_delay_secs'] = event_features['avg_click_delay_secs'].fillna(9999)
event_features[['num_impressions', 'num_clicks', 'ctr']] = event_features[['num_impressions', 'num_clicks', 'ctr']].fillna(0)

In [51]:
event_features

,id3,num_impressions,num_clicks,ctr,avg_click_delay_secs,placement_Mobile_Timeline,placement_OffersTab,placement_Tiles
0,10139,336,47,0.139466,4.692439,17,239,80
1,101460,3277,202,0.061623,11.481268,230,1374,1673
2,10242,47995,313,0.006521,31.791639,4895,25431,17669
3,104246,14157,594,0.041955,46.669261,979,8419,4759
4,106207,968,14,0.014448,36.804567,0,0,968
...,...,...,...,...,...,...,...,...
918,997352,1870,53,0.028327,14.365066,125,957,788
919,99856,20816,967,0.046452,44.712504,1446,12468,6902
920,99877,746,40,0.053548,78.444587,47,484,215
921,998990,1,0,0.000000,9999.000000,0,0,1


In [52]:
event_click_delay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851 entries, 0 to 850
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id3                   851 non-null    object 
 1   avg_click_delay_secs  851 non-null    float64
dtypes: float64(1), object(1)
memory usage: 13.4+ KB


In [53]:
placement_ctr = df_event.copy()
placement_ctr['clicked'] = placement_ctr['id7'].notna().astype(int)

placement_ctr = placement_ctr.groupby(['id3', 'id6'])['clicked'].agg(['count', 'sum']).reset_index()
placement_ctr['placement_ctr'] = placement_ctr['sum'] / (placement_ctr['count'] + 1)

# Pivot to wide format: one row per offer
placement_ctr_wide = placement_ctr.pivot(index='id3', columns='id6', values='placement_ctr').add_prefix('ctr_placement_').reset_index()


In [54]:
df_event['impression_time'] = pd.to_datetime(df_event['id4'], errors='coerce')
df_event['impression_hour'] = df_event['impression_time'].dt.hour
df_event['impression_weekday'] = df_event['impression_time'].dt.weekday


In [55]:
# Average time of day for impressions
avg_time_features = df_event.groupby('id3').agg({
    'impression_hour': 'mean',
    'impression_weekday': 'mean'
}).reset_index().rename(columns={
    'impression_hour': 'avg_impression_hour',
    'impression_weekday': 'avg_impression_weekday'
})


In [56]:
unique_users = df_event.groupby('id3')['id2'].nunique().reset_index(name='num_unique_users')
click_users = df_event[df_event['id7'].notna()].groupby('id3')['id2'].nunique().reset_index(name='num_users_clicked')


In [57]:
df_event['click_time'] = pd.to_datetime(df_event['id7'], errors='coerce')
df_event['click_delay'] = (df_event['click_time'] - df_event['impression_time']).dt.total_seconds()

click_delay_stats = df_event[df_event['click_delay'].notna()].groupby('id3')['click_delay'].agg(['min', 'max', 'std']).reset_index()
click_delay_stats.columns = ['id3', 'min_click_delay', 'max_click_delay', 'std_click_delay']


In [58]:
def bucket_lag(sec):
    if pd.isna(sec): return 'No Click'
    elif sec < 60: return 'Immediate'
    elif sec < 300: return 'Short'
    elif sec < 1800: return 'Medium'
    else: return 'Long'

df_event['click_delay_bucket'] = df_event['click_delay'].apply(bucket_lag)
delay_bucket_count = pd.crosstab(df_event['id3'], df_event['click_delay_bucket']).reset_index()
delay_bucket_count.columns = ['id3'] + ['clicks_' + c.lower().replace(" ", "_") for c in delay_bucket_count.columns[1:]]


In [59]:
event_features = event_ctr.merge(event_click_delay, on='id3', how='left') \
                          .merge(placement_ctr_wide, on='id3', how='left') \
                          .merge(avg_time_features, on='id3', how='left') \
                          .merge(unique_users, on='id3', how='left') \
                          .merge(click_users, on='id3', how='left') \
                          .merge(click_delay_stats, on='id3', how='left') \
                          .merge(delay_bucket_count, on='id3', how='left')

# Fill NaNs appropriately
event_features.fillna({
    'avg_click_delay_secs': 9999,
    'min_click_delay': 9999,
    'max_click_delay': 9999,
    'std_click_delay': 0,
    'num_users_clicked': 0,
    'num_unique_users': 0,
}, inplace=True)


In [60]:
event_features.shape

(923, 20)

In [61]:
# Save to current working directory
offer_features.to_csv("offer_features.csv", index=False)
event_features.to_csv("event_features.csv", index=False)


In [62]:
# Look at overlaps
train_ids = set(df_train['id2'].unique())
trans_ids = set(df_transaction['id2'].unique())

overlap = train_ids & trans_ids
print(f'Overlapping customers: {len(overlap)}')


Overlapping customers: 0


In [63]:
df_transaction.head()
df_transaction.columns.tolist()


['id2', 'f367', 'f368', 'f369', 'f370', 'f371', 'f372', 'id8', 'f374']

In [64]:
df_transaction.head()

,id2,f367,f368,f369,f370,f371,f372,id8,f374
0,2896709,15.60,PBR,D,2023-10-16,19:16:52,202310,59639998,DSE
1,2855047,6.40,PR,D,2023-10-14,13:01:16,202310,59639998,DSE
2,2497175,13.99,PBR,D,2023-10-14,00:31:48,202310,59639998,DSE
3,2655364,15.14,PGC,D,2023-10-13,12:37:25,202310,59639998,DSE
4,2855047,2.12,PR,D,2023-10-09,16:51:21,202310,59639998,DSE


In [65]:
df_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6339465 entries, 0 to 6339464
Data columns (total 9 columns):
 #   Column  Dtype  
---  ------  -----  
 0   id2     int32  
 1   f367    float64
 2   f368    object 
 3   f369    object 
 4   f370    object 
 5   f371    object 
 6   f372    object 
 7   id8     object 
 8   f374    object 
dtypes: float64(1), int32(1), object(7)
memory usage: 411.1+ MB


In [66]:
import pandas as pd

# Step 1: Combine date and time into a single datetime column
df_transaction['datetime'] = pd.to_datetime(df_transaction['f370'] + ' ' + df_transaction['f371'], errors='coerce')

# Step 2: Aggregate transaction-level features by f374 (industry)
trans_by_industry = df_transaction.groupby('f374').agg(
    industry_trans_count=('f367', 'count'),
    industry_avg_amt=('f367', 'mean'),
    industry_std_amt=('f367', 'std'),
    industry_max_amt=('f367', 'max'),
    industry_min_amt=('f367', 'min'),
    unique_products=('f368', 'nunique'),
    unique_customers=('id2', 'nunique'),
    avg_txn_per_customer=('id2', lambda x: x.count() / x.nunique() if x.nunique() > 0 else 0),
    active_days=('datetime', lambda x: x.dt.date.nunique()),
    mode_txn_type=('f369', lambda x: x.mode().iloc[0] if not x.mode().empty else 'NA')
).reset_index()


In [61]:
# Assuming df_offer contains column 'f374' (industry name)
df_offer = df_offer.merge(trans_by_industry, on='f374', how='left')


In [62]:
trans_by_industry.to_csv("trans_by_industry.csv", index=False)


In [63]:
df_offer.head()

,id3,id9,f375,f376,id10,f374,id8,id12,id13,offer_duration_days,...,industry_trans_count,industry_avg_amt,industry_std_amt,industry_max_amt,industry_min_amt,unique_products,unique_customers,avg_txn_per_customer,active_days,mode_txn_type
0,70687,FO5O,2,5.0,1,None,None,2018-01-01 00:00:00,2099-12-31 23:59:59,29949,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,900002526,UGE,2,100.0,1,None,None,2014-10-20 00:00:00,2099-12-31 23:59:59,31118,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,900002864,UTP,1,100.0,1,None,None,2016-07-19 00:00:00,2099-12-31 23:59:59,30480,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,19508,o,2,NaN,1,None,None,2019-06-02 17:00:00,2028-12-31 16:59:59,3499,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,35903,o,2,NaN,1,None,None,2019-06-02 17:00:00,2028-12-31 16:59:59,3499,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
df_offer.to_csv("df_offer1.csv", index=False)


In [65]:
transaction_features = df_transaction.groupby('id2').agg({
    'f367': ['count', 'mean', 'sum', 'max', 'min', 'std'],  # amount stats
    'f368': pd.Series.nunique,  # unique merchant/category
    'f369': pd.Series.nunique,  # unique transaction types
    'f374': pd.Series.nunique,  # unique locations
}).reset_index()

# Rename columns
transaction_features.columns = ['id2', 
    'trans_count', 'trans_amt_mean', 'trans_amt_sum', 'trans_amt_max', 'trans_amt_min', 'trans_amt_std',
    'unique_f368', 'unique_f369', 'unique_f374'
]


In [67]:
print(df_transaction['f371'].unique()[:10])


['19:16:52' '13:01:16' '00:31:48' '12:37:25' '16:51:21' '17:24:37'
 '16:27:55' '20:45:06' '13:10:06' '17:31:55']


In [68]:
# Pad time values to hh:mm:ss if they are in hh:mm format
df_transaction['f371'] = df_transaction['f371'].apply(lambda x: x if len(x.strip().split(':')) == 3 else x + ':00')


In [69]:
df_transaction['f371'] = pd.to_timedelta(df_transaction['f371'], errors='coerce')


In [70]:
df_transaction['f370'] = pd.to_datetime(df_transaction['f370'], errors='coerce')
df_transaction['trans_datetime'] = df_transaction['f370'] + df_transaction['f371']


In [71]:
df_transaction['trans_dayofweek'] = df_transaction['trans_datetime'].dt.dayofweek
df_transaction['trans_hour'] = df_transaction['trans_datetime'].dt.hour
df_transaction['trans_day'] = df_transaction['trans_datetime'].dt.day
df_transaction['trans_month'] = df_transaction['trans_datetime'].dt.month


In [72]:
agg_numeric = df_transaction.groupby('id2').agg({
    'f367': ['mean', 'sum', 'count', 'max', 'min'],
    'trans_hour': ['mean', 'std'],
    'trans_dayofweek': ['nunique'],
})

# Flatten column names
agg_numeric.columns = ['_'.join(col) for col in agg_numeric.columns]
agg_numeric.reset_index(inplace=True)


In [73]:
for col in ['f368', 'f369', 'f374']:
    freq = df_transaction.groupby('id2')[col].agg(['nunique', pd.Series.mode])
    freq.columns = [f'{col}_nunique', f'{col}_mode']
    agg_numeric = agg_numeric.merge(freq, on='id2', how='left')


In [74]:
latest_date = df_transaction['trans_datetime'].max()

df_transaction['days_ago'] = (latest_date - df_transaction['trans_datetime']).dt.days
recent_agg = df_transaction[df_transaction['days_ago'] <= 30].groupby('id2').agg(
    trans_30d_count=('f367', 'count'),
    trans_30d_amount=('f367', 'sum')
).reset_index()

# Merge recent stats
agg_numeric = agg_numeric.merge(recent_agg, on='id2', how='left')


In [75]:
final_transaction_features = transaction_features.merge(agg_numeric, on='id2', how='outer')


In [76]:
final_transaction_features.head()

,id2,trans_count,trans_amt_mean,trans_amt_sum,trans_amt_max,trans_amt_min,trans_amt_std,unique_f368,unique_f369,unique_f374,...,trans_hour_std,trans_dayofweek_nunique,f368_nunique,f368_mode,f369_nunique,f369_mode,f374_nunique,f374_mode,trans_30d_count,trans_30d_amount
0,2000010,12,165.738333,1988.86,625.85,1.00,229.815205,1,1,6,...,6.971805,6,1,PR,1,D,6,BS,11.0,1363.01
1,2000022,1,2284.650000,2284.65,2284.65,2284.65,NaN,1,1,1,...,NaN,1,1,PR,1,D,1,ATS,1.0,2284.65
2,2000023,32,22.277188,712.87,100.00,1.69,25.327353,2,2,8,...,8.288887,6,2,PPC,2,D,8,EP,31.0,711.18
3,2000025,10,77.449000,774.49,250.00,1.50,82.155547,1,1,6,...,8.044322,5,1,PR,1,D,6,GSS,10.0,774.49
4,2000026,29,424.188621,12301.47,1521.10,0.01,402.801807,1,2,6,...,6.448164,6,1,OBR,2,D,6,ATS,24.0,8813.85


In [79]:
print('id3 in df_event:', 'id3' in df_event.columns)
print('id3 in df_transaction:', 'id3' in df_transaction.columns)


id3 in df_event: True
id3 in df_transaction: False


In [81]:
print("df_event 'id3' range:", df_event['id3'].min(), "to", df_event['id3'].max())


df_event 'id3' range: 10139 to 99900


In [86]:
print(df_event.columns)


Index(['id2', 'id3', 'id6', 'id4', 'id7', 'clicked', 'impression_time',
       'click_time', 'click_delay_seconds', 'impression_hour',
       'impression_weekday', 'click_delay', 'click_delay_bucket'],
      dtype='object')


In [87]:
# Use click_time if available, else fallback to impression_time
df_event['event_time'] = pd.to_datetime(df_event['click_time'].fillna(df_event['impression_time']), errors='coerce')

# Get latest offer_id (id3) for each customer (id2)
latest_offer = df_event.sort_values('event_time') \
                       .groupby('id2') \
                       .last() \
                       .reset_index()[['id2', 'id3']]

# Merge this into your final transaction feature set
transaction_features_final = transaction_features.merge(latest_offer, on='id2', how='left')


In [88]:
# Assuming df_events has a timestamp column like 'event_time'
df_event['event_time'] = pd.to_datetime(df_event['event_time'], errors='coerce')

# Get the latest offer per customer
latest_id3 = df_event.sort_values('event_time').groupby('id2').last().reset_index()[['id2', 'id3']]

# Merge
transaction_features_final = transaction_features.merge(latest_id3, on='id2', how='left')


In [89]:
transaction_features_final.head()

,id2,trans_count,trans_amt_mean,trans_amt_sum,trans_amt_max,trans_amt_min,trans_amt_std,unique_f368,unique_f369,unique_f374,id3
0,2000010,12,165.738333,1988.86,625.85,1.00,229.815205,1,1,6,31794
1,2000022,1,2284.650000,2284.65,2284.65,2284.65,NaN,1,1,1,399752
2,2000023,32,22.277188,712.87,100.00,1.69,25.327353,2,2,8,605824
3,2000025,10,77.449000,774.49,250.00,1.50,82.155547,1,1,6,65973
4,2000026,29,424.188621,12301.47,1521.10,0.01,402.801807,1,2,6,NaN
